In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import datasets, models
import torchvision.transforms as transforms
from torchvision.utils import make_grid

from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader

from PIL import Image
from IPython.display import display
import cv2
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import glob
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm

# 결과 확인을 용이하게 하기 위한 코드
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = 'all'

In [6]:
# Define transform options
train_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.RandomCrop(227),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                                     ])
                                      
test_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                                     ])

# Data load
train_data = datasets.CIFAR10('./data', download = True, train = True, transform = train_transform) # len : 50000
test_data = datasets.CIFAR10('./data', download = True, train = False, transform = test_transform) # len : 10000

# Train, valid split
valid_size = 0.15
indices = list(range(len(train_data)))
np.random.shuffle(indices)
split = int(np.floor(valid_size * len(train_data)))

train_idx, valid_idx = indices[split:], indices[:split]
print(len(train_idx))
print(len(valid_idx))

# Define samplers
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# Dataloaders (Combine dataset and sampler)
batch_size = 256
train_dl = torch.utils.data.DataLoader(train_data, batch_size = batch_size, sampler = train_sampler)
val_dl = torch.utils.data.DataLoader(train_data, batch_size = batch_size, sampler = train_sampler)
test_dl = torch.utils.data.DataLoader(test_data, batch_size = batch_size)

Files already downloaded and verified
Files already downloaded and verified
42500
7500


In [7]:
# pip install tensorflow

In [8]:
# pip install tensorboardcolab

In [3]:
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback, TensorBoard
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
from sklearn.preprocessing import LabelBinarizer
from tensorflow.nn import local_response_normalization  # lrn

In [4]:
# modeling(functional API)
input_shape = (224, 224, 3)  # 논문에서 제시된 shape
x = Input(shape = input_shape, name='INPUT')

# CONV
conv1 = Conv2D(filters=96, kernel_size=11, activation='relu', strides=4, name='CONV_1')(x)
pool1 = MaxPooling2D((3,3), strides=2, name='POOL_1')(conv1)  # overlapped pooling
# lrn1 = local_response_normalization(conv1,depth_radius=5, bias=2, alpha=0.0001, beta=0.75) 
lrn1 = BatchNormalization(name='LRN_1')(pool1)

conv2 = Conv2D(filters=256, kernel_size=5, activation='relu', strides=1, padding='same', name='CONV_2')(lrn1)
pool2 = MaxPooling2D((3,3), strides=2, name='POOL_2')(conv2)
# lrn2 = local_response_normalization(conv2,depth_radius=5, bias=2,  alpha=0.0001, beta=0.75)
lrn2 = BatchNormalization(name='LRN_2')(pool2)

conv3 = Conv2D(filters=384, kernel_size=3, activation='relu', strides=1, padding='same', name='CONV_3')(lrn2)
conv4 = Conv2D(filters=384, kernel_size=3, activation='relu', strides=1, padding='same', name='CONV_4')(conv3)
conv5 = Conv2D(filters=256, kernel_size=3, activation='relu', strides=1, padding='same', name='CONV_5')(conv4)
pool3 = MaxPooling2D((3,3), strides=2, name='POOL_3')(conv5)

# FC
f = Flatten()(pool3)
f = Dense(4096, activation='relu', name='FC_1')(f)
f = Dropout(0.5)(f)  # 논문 parameter 0.5 이용
f = Dense(4096, activation='relu', name='FC_2')(f)
f = Dropout(0.5)(f)
out = Dense(1000, activation='softmax', name='OUTPUT')(f)

model = Model(inputs=x, outputs=out)
model.summary()

2022-04-14 16:47:37.563079: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 INPUT (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 CONV_1 (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 POOL_1 (MaxPooling2D)       (None, 26, 26, 96)        0         
                                                                 
 LRN_1 (BatchNormalization)  (None, 26, 26, 96)        384       
                                                                 
 CONV_2 (Conv2D)             (None, 26, 26, 256)       614656    
                                                                 
 POOL_2 (MaxPooling2D)       (None, 12, 12, 256)       0         
                                                                 
 LRN_2 (BatchNormalization)  (None, 12, 12, 256)       1024  

In [5]:
# cifar10 data load
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

170508288/170498071 [==============================] - 2143s 13us/step


In [8]:
x_train.shape

(50000, 32, 32, 3)

In [15]:
import numpy as np

## 227, 227로 Resize 시도 -> RAM 초과 
from keras.datasets import cifar10
import cv2
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

data_upscaled = np.zeros((50000, 3, 227, 227))
for i, img in enumerate(x_train):
    im = img.reshape((1, 2, 0))
    large_img = cv2.resize(im, dsize=(227, 227), interpolation=cv2.INTER_CUBIC)
#     large_img = cv2.resize(x_train, dsize=(227, 227), interpolation=cv2.INTER_AREA)
    data_upscaled[i] = large_img.reshape((2, 0, 1))

ValueError: cannot reshape array of size 3072 into shape (1,2,0)

In [6]:
batch_size = 128
num_classes = 10
epochs = 10

y_train = to_categorical(y_train, num_classes)  # category화 
y_test = to_categorical(y_test, num_classes)

model = Sequential()
# model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', padding='same', input_shape=(img_height, img_width, channel,)))
# for original Alexnet
model.add(Conv2D(48, (3,3), strides=(2,2), activation='relu', padding='same', input_shape=(img_height, img_width, channel,)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Conv2D(96, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Conv2D(192, (3,3), activation='relu', padding='same'))
model.add(Conv2D(192, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(512, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10, 
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
391/391 [==============================] - 37s 93ms/step - loss: 1.5531 - accuracy: 0.4523 - val_loss: 1.5336 - val_accuracy: 0.4825
Epoch 2/10
391/391 [==============================] - 34s 87ms/step - loss: 1.0999 - accuracy: 0.6188 - val_loss: 1.0576 - val_accuracy: 0.6314
Epoch 3/10
391/391 [==============================] - 35s 90ms/step - loss: 0.9168 - accuracy: 0.6855 - val_loss: 1.0726 - val_accuracy: 0.6298
Epoch 4/10
391/391 [==============================] - 34s 87ms/step - loss: 0.7964 - accuracy: 0.7284 - val_loss: 0.9223 - val_accuracy: 0.6878
Epoch 5/10
391/391 [==============================] - 33s 84ms/step - loss: 0.6984 - accuracy: 0.7649 - val_loss: 1.0117 - val_accuracy: 0.6637
Epoch 6/10
391/391 [==============================] - 34s 88ms/step - loss: 0.6204 - accuracy: 0.7933 - val_loss: 0.9693 - val_accuracy: 0.6882
Epoch 7/10
391/391 [==============================] - 35s 89ms/step - loss: 0.5401 - accuracy: 0.8203 - val_loss: 1.0505 - val_accuracy: